In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.layers import LSTM
from keras.backend import argmax
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.utils import to_categorical

ImportError: No module named 'keras'

In [2]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        #n=round((i / 6) % 24)
        n=((i) / 6) % 24
        #n=n/24
        dataY.append(n)
    return numpy.array(dataX), numpy.array(dataY)

In [ ]:
Pdataset = pandas.read_csv('piecstyczen.csv', usecols=[2,3], engine='python', skipfooter=3)
plt.figure(figsize=(15,3))
plt.plot(Pdataset)
plt.show()

In [ ]:
Pdataset.tail(4)

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(Pdataset)

In [ ]:
plt.plot(dataset)
plt.show()

In [ ]:
dataset

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# reshape into X=t and Y=t+1
look_back = 60
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
train.shape

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
print(trainX.shape)
#dataset.shape
print(trainY.shape)
print(train.shape)

In [ ]:
plt.figure(figsize=(15,3))

fig, ax1 = plt.subplots()


ax2 = ax1.twinx()
ax1.plot(test)

ax2.plot(testY,color="red")
plt.show()

In [ ]:
trainY


In [ ]:
#trainX.reshape(-1,4)

In [ ]:
# create and fit the LSTM network
#https://github.com/ar-ms/lstm-mnist/blob/master/lstm_classifier.py
model = Sequential()
model.add(LSTM(105,  input_shape=(1, look_back)))
#model.add(Dense(30,activation="relu",input_dim=30))
model.add(Dropout(0.02))
#model.add(Dense(30,activation="relu"))
model.add(Dense(25,activation="softmax"))
#model.add(Dense(32, input_shape=(20,)))
#model.add(Activation('relu'))
#model.add(Dense(25, activation='relu'))
#model.compile(loss='mean_squared_error', optimizer='rmsprop')
trainYO = to_categorical(trainY, num_classes=25)
model.compile(loss='categorical_crossentropy',      optimizer='rmsprop',     metrics=['accuracy'])
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

### Train the residual compressor
earlystopper = EarlyStopping(patience=5, verbose=1) 
checkpointer = ModelCheckpoint('VSB_classifier', verbose=1, save_best_only=True)
#results = class_model.fit(X, y, validation_split=0.2, batch_size=50, epochs=300, 
#                    callbacks=[earlystopper, checkpointer])
model.fit(trainX, trainYO, epochs=100, batch_size=1, verbose=2,callbacks=[earlystopper, checkpointer])

In [ ]:
score = model.evaluate(trainX, trainYO, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
# make predictions
trainPredict1 = model.predict(trainX)
testPredict1 = model.predict(testX)
trainPredict1

In [ ]:
#softmax(trainPredict)
trainPredict=numpy.argmax(trainPredict1,1)
testPredict=numpy.argmax(testPredict1,1)
trainPredict

In [ ]:
# invert predictions
#trainPredict = scaler.inverse_transform(trainPredict)
#trainY = scaler.inverse_transform([trainY])
#testPredict = scaler.inverse_transform(testPredict)
#testY = scaler.inverse_transform([testY])

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.3f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.3f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
print(trainPredictPlot.shape)
trainPredictPlot[look_back:len(trainPredict)+look_back, 0] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)-7:len(dataset)-9, 0] = testPredict
# plot baseline and predictions
#plt.plot(scaler.inverse_transform(dataset),label="set")
plt.plot(trainPredictPlot,label="train")
plt.plot(testPredictPlot,label="test")
plt.legend()
plt.show()

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
wws=2;
trainPredictPlot[wws:len(trainPredict)+wws,0] = trainPredict
wws=60
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+wws+1:len(dataset)-look_back-1, 0] = testPredict
# plot baseline and predictions

fig, ax1 = plt.subplots()


ax2 = ax1.twinx()
ax1.plot(trainY,label="set")
ax1.plot(trainPredictPlot,label="train")
ax2.plot(testPredictPlot,label="testm")
#plt.figure(figsize=(15,3))

plt.legend()
plt.show()